In [ ]:
# 1. 환경 설정 

import argparse 
import collections 
import os 
import random 
import re 
import shutil 
import zipfile 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import torch 

In [ ]:
# kowiki.txt.zip 파일 다운 받아 구글 폴더 업로드 하기 
!wget https://easy-server.s3.ap-northeast-2.amazonaws.com/kowiki.txt.zip o

--2023-05-01 01:15:02--  https://easy-server.s3.ap-northeast-2.amazonaws.com/kowiki.txt.zip
Resolving easy-server.s3.ap-northeast-2.amazonaws.com (easy-server.s3.ap-northeast-2.amazonaws.com)... 52.219.56.71
Connecting to easy-server.s3.ap-northeast-2.amazonaws.com (easy-server.s3.ap-northeast-2.amazonaws.com)|52.219.56.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228833965 (218M) [application/zip]
Saving to: ‘kowiki.txt.zip.1’

kowiki.txt.zip.1    100%[===================>] 218.23M  40.2MB/s    in 6.0s    

2023-05-01 01:15:08 (36.6 MB/s) - ‘kowiki.txt.zip.1’ saved [228833965/228833965]

--2023-05-01 01:15:08--  http://o/
Resolving o (o)... failed: Name or service not known.
wget: unable to resolve host address ‘o’
FINISHED --2023-05-01 01:15:08--
Total wall clock time: 6.4s
Downloaded: 1 files, 218M in 6.0s (36.6 MB/s)


In [ ]:
kowiki_file = './kowiki.txt.zip'

In [ ]:
# 환경 설정 

args ={ 
    # random seed value 
    "seed": 1234,
    # cpu 또는 gpu 사용 여부 결정 
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    # 말뭉치 파일 
    "corpus": kowiki_file,
}

args = argparse.Namespace(**args)

In [ ]:
print(args)

Namespace(seed=1234, device=device(type='cuda'), corpus='./kowiki.txt.zip')


In [ ]:
# random seed 설정 

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

In [ ]:
# 파일 내용 확인 

# wiki lines (라인 수) 확인 

count = 0 
with zipfile.ZipFile(args.corpus) as z: 
  with z.open('kowiki.txt') as f:
    for i, line in enumerate(f):
      count += 1

print(count)

3724301


In [ ]:
# wiki 내용 확인 

with zipfile.ZipFile(args.corpus) as z: 
  with z.open('kowiki.txt') as f: 
    for i, line in enumerate(f):
      if i >= 50: 
        break 
      line  = line.decode('utf-8').strip()
      print(line)

지미 카터
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.
1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다.
카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.
카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다.
그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 1979년 백악관에서 양국 간의 평화조약으로 이끌어졌다. 또한 소련과 제2차 전략 무기 제한 협상에 조인했다.
카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.
그러나 주 이란 미국 대사관 인질 사건에서 인질 구출 실패를 이유로 1980년 대통령 선거에서 공화당의 로널드 레이건 후보에게 져 결국 재선에 실패했다. 또한 임기 말기에 터진 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들의 보이콧을 내세웠다.
지미 카터는 대한민국과의 관계에서도 중요한 영향을 미쳤던 대통

In [ ]:
# Sentencepiece 
# 문장을 일정한 크기 subword 분할 기술 
# "안녕하세요" >> "안녕", "하세요"
# 언어에 무관, 띄어쓰기 유무에 영향 없음 
# 공백 문자 >> 심볼로 취급 (이를 구분하기 위해 공백 문자를 특정 심볼로 변환)

In [ ]:
# sentencepiece 설치 
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00


In [ ]:
import sentencepiece as spm 

In [ ]:
def train_sentencepiece(corpus, prefix, vocab_size = 32000): 
  '''
  sentencepiece 이용, vocab 학습 
  : param corpus : 학습할 말뭉치
  : param prefix : 저장할 vocab 이름 
  : param vocab_size : vocab 개수 
  '''
  spm.SentencePieceTrainer.train(
      f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + # 7은 특수문자 개수
       "--model_type=unigram"+           # 단어 하나씩 
       "--max_sentence_length=999999"+     # 문장 최대 길이
       "--pad_id=0 --pad_piece=[PAD]"+     # pad token 및 id 지정 
       "--unk_id=1 --unk_piece=[UNK]"+     # OOV(unknown token) 및 id 지정 
       "--bos_id=2 --bos_piece=[BOS]"+     # begin of sequence token 및 id 지정
       "--eos_id=3 --eos_piece=[EOS]"+     # end of sequnece token 및 id 지정 
       "--user_defined_symbols=[SEP],[CLS],[MASK]"+ 
       # 기타 추가 토근 SEP:4, CLS: 5, MASK: 6
       "--input_sentence_size=20000"+      # 말뭉치에서 샘플링해서 학습 
       "--shuffle_input_sentence=True"     # 샘플링한 말뭉치 shuffle 
  )

In [ ]:
kowiki_file

'./kowiki.txt.zip'

In [ ]:
!unzip './kowiki.txt.zip'

Archive:  ./kowiki.txt.zip
replace kowiki.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
print(os.listdir('./'))

['.config', 'kowiki.txt.zip.1', 'kowiki.txt.zip', '__MACOSX', 'kowiki.txt', 'sample_data']


In [ ]:
# vocab 학습 

train_sentencepiece('kowiki.txt', 'kowiki_32000')
# voca size 를 32000 인 파일 생성 

In [ ]:
# 생성 결과 확인 
print(os.listdir('./'))

['.config', 'kowiki.txt.zip.1', 'kowiki.txt.zip', 'kowiki_32000.model', '__MACOSX', 'kowiki.txt', 'kowiki_32000.vocab', 'sample_data']


In [ ]:
# 생성파일 백업 

corpus_dir = os.path.dirname(args.corpus)

print(os.listdir(corpus_dir))

['.config', 'kowiki.txt.zip.1', 'kowiki.txt.zip', 'kowiki_32000.model', '__MACOSX', 'kowiki.txt', 'kowiki_32000.vocab', 'sample_data']


In [ ]:
# load vocab 

spm_vocab = spm.SentencePieceProcessor()
spm_vocab.load(os.path.join(corpus_dir, 'kowiki_32000.model'))

True

In [ ]:
# wiki spm tokenize 

with zipfile.ZipFile(args.corpus) as z :
  with z.open('kowiki.txt') as f: 
    for i, line in enumerate(f):
      if i >= 5: 
        break 
      line = line.decode('utf-8').strip()
      print(line)
      tokens = spm_vocab.encode_as_pieces(line)
      print(tokens)
      _ids = spm_vocab.encode_as_ids(line)
      print(_ids)

지미 카터
['▁지미', '▁카터']
[12698, 12747]
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
['▁제임스', '▁얼', '▁"', '지', '미', '"', '▁카터', '▁주니어', '(', ',', '▁1924', '년', '▁10', '월', '▁1', '일', '▁~', '▁', ')', '는', '▁민주당', '▁출신', '▁미국', '▁39', '번', '째', '▁대통령', '▁', '(1977', '년', '▁~', '▁1981', '년', ')', '이다', '.']
[1885, 5687, 57, 28, 141, 42, 12747, 5039, 10, 4, 4107, 11, 93, 21, 37, 24, 110, 6, 9, 8, 2186, 1078, 164, 4677, 200, 579, 496, 6, 20825, 11, 110, 1888, 11, 9, 29, 3]
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
['▁지미', '▁카터', '는', '▁조지아주', '▁섬', '터', '▁카운티', '▁', '플레인', '스', '▁마을에서', '▁태어났다', '.', '▁조지아', '▁공과대학교', '를', '▁졸업하였다', '.', '▁그', '▁후', '▁해군', '에', '▁들어가', '▁전함', '·', '원자력', '·', '잠수함', '의', '▁승무원', '으로', '▁', '일', '하였다', '.', '▁1953', '년', '▁미국', '▁해군', '▁대', '위', '로', '▁예편', '하였고', '▁이후', '▁땅콩', '·', '면', '화', '▁등을', '

In [ ]:
# 문자열을 token으로 변경 

spm_vocab.encode_as_pieces("아름다운 대한민국 우리나라 금수강산")

['▁아름다운', '▁대한민국', '▁우리나라', '▁금', '수', '강', '산']

In [ ]:
spm_vocab.encode_as_pieces("우리는 닥터윌의 수제자, 데싸다")

['▁우리는', '▁닥터', '윌', '의', '▁수', '제', '자', ',', '▁데', '싸', '다']

In [ ]:
tokens = spm_vocab.encode_as_pieces("우리는 닥터윌의 수제자, 데싸다")

In [ ]:
# tokens 을 문자열로 복원 
print(spm_vocab.decode_pieces(tokens))

우리는 닥터윌의 수제자, 데싸다


In [ ]:
# 문자열을 숫자로 분할 
spm_vocab.encode_as_ids("아름다운 대한민국 우리나라 금수강산")

[4334, 254, 5314, 616, 64, 279, 117]

In [ ]:
ids = spm_vocab.encode_as_ids("우리는 닥터윌의 수제자, 데싸다")
print(ids)

[6305, 4816, 21954, 5, 30, 92, 38, 4, 249, 9754, 41]


In [ ]:
# 숫자를 문자열로 복원 
print(spm_vocab.decode_ids(ids))

우리는 닥터윌의 수제자, 데싸다


In [ ]:
# token을 숫자로 변경 
print(spm_vocab.piece_to_id(tokens))

[6305, 4816, 21954, 5, 30, 92, 38, 4, 249, 9754, 41]


In [ ]:
# 숫자를 token으로 변경 
print(spm_vocab.id_to_piece(ids))

['▁우리는', '▁닥터', '윌', '의', '▁수', '제', '자', ',', '▁데', '싸', '다']
